## SpeechRecognition Python library

O reconhecimento automático de voz é um grande desafio. 

Não faltam empresas e instituições de pesquisa trabalhando em bibliotecas para ajudar a resolvê-lo. 

Há a biblioteca Sphinx da Carnegie Mellon University, Kaldi, SpeechRecognition e muito mais. 

Alguns têm recursos mais robustos do que outros, mas todos têm o mesmo objetivo de transcrever arquivos de áudio para texto. 

Vamos nos concentrar na biblioteca SpeechRecognition por causa de sua baixa barreira de entrada e sua compatibilidade com muitas APIs de reconhecimento de voz disponíveis que veremos em breve.

Para acessar a classe Recognizer, primeiro importaremos o módulo SpeechRecognition como a abreviatura sr. Em seguida, criaremos uma instância da classe Recognizer chamando-a de sr e atribuindo a uma variável, recognizer. 

Por fim, definiremos o limite de energy_threshold para 300. O limite energy_threshold pode ser considerado como o volume do áudio, que é considerado fala. 

Valores abaixo do limite são considerados silêncio, valores acima são considerados fala. Uma sala silenciosa é normalmente entre 0 e 100. 

A documentação da SpeechRecognition recomenda 300 como um valor inicial que cobre a maioria dos arquivos de fala. O valor do limite de energia será ajustado automaticamente conforme o reconhecedor escuta um arquivo de áudio.

In [5]:
import wave
import speech_recognition as sr

In [11]:
audio_file = sr.AudioFile('dataset/clean-support-call.wav')

recognizer = sr.Recognizer()

recognizer.energy_threshold = 300

with audio_file as source:
    audio_file = recognizer.record(source)

text = recognizer.recognize_google(audio_file,
                           language = "en-US")

In [12]:
text

"hello I'd like to get some help setting up my account please"

## Reading audio files with SpeechRecognition

Felizmente, a biblioteca SpeechRecognition tem uma classe interna, AudioFile, junto com outro método útil na classe Recognizer, record. 

Podemos usá-los para cuidar do pré-processamento para nós. 

Para começar, importamos a biblioteca SpeechRecognition e instanciamos uma instância do recognizer como antes. Então, para ler nosso arquivo de áudio, acessamos a classe AudioFile e passamos nosso nome de arquivo de áudio e o salvamos em uma variável. 

Nesse caso, nossa variável AudioFile é chamada de chamada de clean_support_call. Agora, se verificarmos o tipo de chamada de suporte limpa, podemos ver que é uma instância de AudioFile.

In [13]:
import speech_recognition as sr

In [15]:
clean_support_call = sr.AudioFile('dataset/clean-support-call.wav')
recognizer = sr.Recognizer()

In [16]:
type(clean_support_call)

speech_recognition.AudioFile

Nossa variável clean_support_call é atualmente do tipo AudioFile. 

Para convertê-la para o tipo de dados de áudio, podemos usar o método source da classe do recognizer. 

* Usamos um gerenciador de contexto, também conhecido como with, para abrir e ler o arquivo de áudio clean_support_call como source. 
* Em seguida, criamos o clean_support_call_audio usando o método record e transmitindo-o a source. 
* Agora, antes de chamarmos o recognizer do google novamente, vamos verificar o tipo de clean_support_call_audio.

In [19]:
with clean_support_call as source:
    clean_support_call_audio = recognizer.record(source)
    
type(clean_support_call_audio)

speech_recognition.AudioData

In [20]:
text = recognizer.recognize_google(audio_data=clean_support_call_audio,
                           language = "en-US")
text

"hello I'd like to get some help setting up my account please"

In [21]:
'''

with clean_support_call as source:
    clean_support_call_audio = recognizer.record(source,
                                                duration=None,
                                                offset = None) 
    
with clean_support_call as source:
    clean_support_call_audio = recognizer.record(source,
                                                duration=2.0) 
                                                
'''

'\n\nwith clean_support_call as source:\n    clean_support_call_audio = recognizer.record(source,\n                                                duration=None,\n                                                offset = None) \n    \nwith clean_support_call as source:\n    clean_support_call_audio = recognizer.record(source,\n                                                duration=2.0) \n                                                \n'

## Dealing with different kinds of audio

Embora SpeechRecognition seja capaz de transcrever áudio, ele não sabe necessariamente que tipo de áudio está transcrevendo. Por exemplo, se você passar ao reconhecedor um arquivo de áudio com fala em japonês, mas a tag de idioma for inglês dos EUA, você receberá de volta o áudio em japonês em caracteres do inglês.

In [22]:
import speech_recognition as sr

In [23]:
japanese_gm = sr.AudioFile('dataset/good-morning-japanense.wav')
recognizer = sr.Recognizer()

In [24]:
with japanese_gm as source:
    japanese_gm_audio = recognizer.record(source)

In [25]:
text = recognizer.recognize_google(audio_data=japanese_gm_audio,
                           language = "en-US")
text

'ohayo gozaimasu'

E se você passar o mesmo arquivo de áudio com a tag de idioma definida como ja para japonês, você obterá o áudio transcrito em caracteres japoneses. 

É importante observar que a biblioteca SpeechRecognition não detecta idiomas automaticamente. Portanto, você terá que garantir que esse parâmetro seja definido manualmente e que a API que você está usando tenha a capacidade de transcrever o idioma em que seus arquivos de áudio estão.

In [26]:
text = recognizer.recognize_google(audio_data=japanese_gm_audio,
                           language = "ja")
text

'おはようございます'

Se passarmos ao SpeechRecogition um arquivo de áudio de um leopardo rugindo, ele retornará um erro de valor desconhecido porque nenhuma fala foi detectada. 

O que também faz sentido porque o rugido de um leopardo, embora muito legal, não é fala humana.

Podemos evitar erros usando o parâmetro show all. O parâmetro show all mostra uma lista de todas as possíveis transcrições da função recognizer. No caso de nosso rugido de leopardo, a lista volta vazia, mas evitamos gerar um erro. No caso do nosso arquivo japonês, você pode ver as diferentes transcrições possíveis.

In [27]:
leopard_roar = sr.AudioFile('dataset/leopard.wav')
recognizer = sr.Recognizer()


with leopard_roar as source:
    leopard_roar_audio = recognizer.record(source)    

In [29]:
text = recognizer.recognize_google(leopard_roar_audio,show_all=True)
text

[]

In [31]:
text = recognizer.recognize_google(japanese_gm_audio,
                                   language = "en-US",
                                   show_all=True)
print(text)

{'alternative': [{'transcript': 'ohayo gozaimasu', 'confidence': 0.98762906}], 'final': True}


Em seguida, caso o audio contenha varias pessoas falando ao mesmo tempo. 

A API da Web gratuita do Google transcreve a fala e a retorna como um único bloco de texto, não importa quantos falantes haja. 

Um único bloco de texto retornado ainda pode ser útil, no entanto, se o seu problema requer saber quem disse o quê, você pode considerar a API gratuita que estamos usando apenas como uma prova de conceito. Em seguida, use uma das versões pagas para tarefas mais complexas. 

O processo de separar mais de um locutor de um único arquivo de áudio é denominado diarização do locutor; no entanto, está além do escopo deste estudo.


Para contornar o problema de vários falantes, você pode garantir que seus arquivos de áudio sejam gravados separadamente para cada falante. Em seguida, transcreva o áudio dos alto-falantes individuais.

In [33]:
multiple_speakers = sr.AudioFile('dataset/multiple-speakers-16k.wav')
recognizer = sr.Recognizer()

with multiple_speakers as source:
    multiple_speakers_audio = recognizer.record(source)
    
text = recognizer.recognize_google(multiple_speakers_audio, language = "en-US")
text

"is that it doesn't recognize different speakers invoices it will just return it all as one block of text"

In [ ]:
speakers = [sr.AudioFile('dataset/s0.wav'), sr.AudioFile('dataset/s1.wav'), sr.AudioFile('dataset/s2.wav')]

for i, speaker in enumerate(speakers):
    with speakers as source:
        speaker_audio = recognizer.record(source)
    print(f"Text from speaker {i}: {recognizer.recognize_google(speaker_audio)}")

Finalmente, há o problema do ruído de fundo. 

Uma regra prática a ser lembrada é que, se você tiver problemas para entender o que está sendo dito em um arquivo de áudio devido ao ruído de fundo, é provável que um sistema de reconhecimento de voz também tenha. 

Para tentar acomodar o ruído de fundo, a classe recognizer tem uma função embutida, ajuste para ruído ambiente, que leva um parâmetro, duração. 

A classe do reconhecedor então escuta por segundos de duração no início do arquivo de áudio e ajusta o limite de energia, ou a quantidade que a classe do reconhecedor escuta, para um nível adequado para o ruído de fundo. Quanto espaço você tem no início do seu arquivo de áudio irá ditar como você pode definir o valor da duração. A documentação da SpeechRecognition recomenda algo entre zero vírgula cinco a um segundo como um bom ponto de partida.

In [34]:
noisy_support_call = sr.AudioFile('dataset/2-noisy-support-call.wav')
recognizer = sr.Recognizer()

with noisy_support_call as source:
    recognizer.adjust_for_ambient_noise(source, duration=0.5)
    noisy_support_call_audio = recognizer.record(source)
    
recognizer.recognize_google(noisy_support_call_audio)    

"hello I'd like to get to help setting up my account"